# Libraries, Imports, and Data Reading

In [3]:
import json
import zipfile
import re
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
#from google.colab import drive

In [ ]:
# User Toggle
curr_user = 'Ruby_Local'

if curr_user == 'Ruby_Local':
  with open("C:/Users/rubyc/Documents/work/cjlc/Amador/scraped_data_amador_09_23_version/scraped_data.json") as file:
    data = json.load(file)
elif curr_user == 'Ruby_Colab':
  drive.mount('/content/drive', force_remount=True)
  with open("/content/drive/MyDrive/Work/CLJC/scraped_data.json") as file:
    data = json.load(file)

# Parsing

## New Parse

In [5]:
# Helper function to flatten nested fields
def flatten_nested_field(field, default=""):
    if isinstance(field, list):
        flat_list = []
        for item in field:
            if isinstance(item, list):
                flat_list.extend(item)
            elif item:
                flat_list.append(item)
        return "; ".join(flat_list) if flat_list else default
    return str(field) if field else default

# Initialize lists for each component
caseinfo_data = []
parties_data = []
events_data = []
charges_data = []
documents_data = []
bail_data = []
probation_data = []

# Loop through each case entry
for case_id, case_entries in data.items():
    for entry in case_entries:
        # Extract top-level case information
        case_info = {
            "case_id": case_id,
            "case_name": entry.get("Case Name", ""),
            "filing_date": entry.get("Filing Date", ""),
            "location": entry.get("Location", ""),
            "case_type": entry.get("Case Type", ""),
            "category": entry.get("Category", ""),
            "status": entry.get("Status", "")
        }
        caseinfo_data.append(case_info)

        # Process Parties
        parties = entry.get("Parties", {}).get("Parties", [])
        for party in parties:
            party_type = flatten_nested_field(party.get("Type"))
            party_name = flatten_nested_field(party.get("Name"))
            represented_by = flatten_nested_field(party.get("Represented By"))

            if party_type or party_name or represented_by:
                parties_data.append({
                    **case_info,
                    "type": party_type,
                    "name": party_name,
                    "represented_by": represented_by
                })

        # Process Events
        events_list = entry.get("Events", {}).get("Events", [])
        for event in events_list:
            event_data = {
                **case_info,
                "date_time": flatten_nested_field(event.get("Date/Time")),
                "type": flatten_nested_field(event.get("Type")),
                "result": flatten_nested_field(event.get("Result")),
                "official": flatten_nested_field(event.get("Official")),
                "location": flatten_nested_field(event.get("Location"))
            }
            events_data.append(event_data)

        # Process Documents
        documents = entry.get("Documents", {}).get("Documents", [])
        for document in documents:
            document_data = {
                **case_info,
                "filed_date": flatten_nested_field(document.get("Filed")),
                "description": flatten_nested_field(document.get("Description")),
                "filed_by": flatten_nested_field(document.get("Filed By")),
                "tracking_date": flatten_nested_field(document.get("Tracking Date"))
            }
            documents_data.append(document_data)

        # Process Charges
        charges = entry.get("Filings", {}).get("Charges", [])
        for charge in charges:
            charge_data = {
                **case_info,
                "case_name": flatten_nested_field(charge.get("Charge Information")),
                "disposition": flatten_nested_field(charge.get("Disposition"))
            }
            charges_data.append(charge_data)

        # Process Bail Bonds
        bail_bonds = entry.get("Filings", {}).get("Bail Bonds", [])
        for bail_entry in bail_bonds:
            bail_data.append({
                **case_info,
                "bail_type_status": flatten_nested_field(bail_entry.get("Type/Status")),
                "bail_date": flatten_nested_field(bail_entry.get("Date")),
                "bail_posted_by": flatten_nested_field(bail_entry.get("Posted By"))
            })

        # Process Probation
        probation_entries = entry.get("Filings", {}).get("Probation", [])
        for probation_entry in probation_entries:
            conditions_data = flatten_nested_field(probation_entry.get("Type & Conditions", []))

            # Split by ";" and strip spaces
            items = [i.strip() for i in conditions_data.split(";") if i.strip()]

            # First item = probation_type, rest = probation_conditions
            probation_type = items[0] if items else ""
            probation_conditions = "; ".join(items[1:]) if len(items) > 1 else ""

            probation_data.append({
                **case_info,
                "probation_type": probation_type,
                "probation_conditions": probation_conditions
            })

# Convert lists to DataFrames
cases = pd.DataFrame(caseinfo_data)
parties = pd.DataFrame(parties_data)
events = pd.DataFrame(events_data)
charges = pd.DataFrame(charges_data)
documents = pd.DataFrame(documents_data)
bail = pd.DataFrame(bail_data)
probation = pd.DataFrame(probation_data)

# Cleaning

### General Use Functions

In [6]:
def drop_empty(df):
  df.replace(to_replace={"": np.nan}, inplace=True)
  df.dropna(how='all', axis=1, inplace=True)
  df.reset_index(drop=True, inplace=True)
  return df

##Cases

In [7]:
# Remove duplicate rows based on 'Case ID'
cases = cases.drop_duplicates(subset='case_id')
cases = cases.copy()

# Extract dates from 'status' and store them in 'update_date'
cases['update_date'] = cases['status'].str.extract(r'(\d{2}/\d{2}/\d{4})', expand=False)

# Normalize and sanitize 'status' values by removing newlines, dates, and converting to lowercase
cases['status'] = (
    cases['status']
    .str.replace('\n', ' ', regex=True)
    .str.replace(r'\d{2}/\d{2}/\d{4}', '', regex=True)
    .str.strip()
    .str.lower()
)

def map_status(s):
    s = s.lower().strip()  # Normalize input

    # 1. Open
    if s == 'open':
        return 'Open'

    # 2. Conviction — Nolo Plea
    elif 'nolo plea' in s:
        return 'Conviction — Nolo Plea'

    # 3. Conviction — Guilty
    elif ('guilt' in s or
          'jury verdict' in s or
          'court finding of guilt' in s or
          'guilty plea' in s or
          'found true' in s or
          'admitted' in s or
          'bail forfeiture' in s or
          'collections conviction: guilty plea' in s):
        return 'Conviction — Guilty'

    # 4. Acquitted
    elif ('acquittal' in s or
          'found not true' in s or
          'ng/reason of insanity' in s):
        return 'Acquitted'

    # 5. Dismissed
    elif 'dismissal' in s:
        return 'Dismissed'

    # 6. Disposed
    elif ('disposed' in s or
          'consolidated' in s or
          'collections case disposed' in s or
          'transfer out' in s):
        return 'Disposed'

    # 7. Other
    elif ('intracounty transfer' in s or
          'transfer in' in s or
          'extradition waived' in s or
          'petition granted' in s or
          'petition denied' in s or
          'petition summarily denied' in s):
        return 'Other'

# Map statuses to predefined categories
cases['status'] = cases['status'].apply(map_status)

# Make all dates datetime and coerce errors
cases['update_date'] = pd.to_datetime(cases['update_date'], errors='coerce')
cases['filing_date'] = pd.to_datetime(cases['filing_date'], errors='coerce')

In [8]:
cases.head()

,case_id,case_name,filing_date,location,case_type,category,status,update_date
0,24CR34186,People vs. YESSENIA ESCOBAR ROCHA,2024-09-18,Criminal/Traffic,Misdemeanor,Misdemeanor: Drug Offense,Open,NaT
2,24CR34185,"People vs. CESAR JUNIOR MORENO, Jr",2024-09-18,Criminal/Traffic,Misdemeanor,Misdemeanor: DUI,Open,NaT
4,24CR34184,People vs. COURTNEY JONATHAN GARLAND,2024-09-18,Criminal/Traffic,Misdemeanor,Misdemeanor: Non Traffic/Ordinance,Open,NaT
6,24CR34183,People vs. BENJAMIN WILSON HOLCOMB,2024-09-17,Criminal/Traffic,Misdemeanor,Misdemeanor: Traffic Other,Open,NaT
8,24CR34182,People vs. JOSHUA JAMES BANNON,2024-09-17,Criminal/Traffic,Misdemeanor,Misdemeanor: 14601VC,Open,NaT


## Parties

In [ ]:
# Handle NA Columns
parties = drop_empty(parties)

# Normalize 'represented_by' column for "IN PRO PER"
parties['represented_by'] = parties['represented_by'].replace(to_replace=r".*\bIN PRO PER\b.*", value="IN PRO PER", regex=True)

# Drop empty columns
parties.dropna(how='all', axis=1, inplace=True)

#Normalize Name
parties['name'] = parties['name'].apply(lambda x: re.sub(r"\n.*", "", str(x)).strip())


C:\Users\rubyc\AppData\Local\Temp\ipykernel_2448\1030528905.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace(to_replace={"": np.nan}, inplace=True)


In [10]:
parties.head()

,case_id,type,name,represented_by
0,24CR34186,Defendant,"ROCHA, YESSENIA ESCOBAR",NaN
1,24CR34185,Defendant,"MORENO, CESAR JUNIOR Jr",NaN
2,24CR34184,Defendant,"GARLAND, COURTNEY JONATHAN",NaN
3,24CR34183,Defendant,"HOLCOMB, BENJAMIN WILSON",NaN
4,24CR34182,Defendant,"BANNON, JOSHUA JAMES",NaN


## Events

In [11]:
# Handle NA Columns
events = drop_empty(events)

# Remove empty duplicate rows
events = events[~(events['location'].isna() & events['type'].isna() & events['official'].isna() & events['date_time'].isna())]

# Remove case name from Date/Time and rename to Date
events['date_time'] = events['date_time'].str.replace(r';.*', '', regex=True)
events.rename(columns={'date_time': 'date'}, inplace=True)
events['date'] = pd.to_datetime(events['date'], errors='coerce')


C:\Users\rubyc\AppData\Local\Temp\ipykernel_2448\1030528905.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace(to_replace={"": np.nan}, inplace=True)
C:\Users\rubyc\AppData\Local\Temp\ipykernel_2448\72661210.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  events['date'] = pd.to_datetime(events['date'], errors='coerce')


In [12]:

#motion results mapping
def map_motion_results(s):
  #try:
    if s is not None:
        s = str(s)
        s = s.lower().strip()  # Normalize input

        # 1. Not Heard
        if 'not heard' in s:
            return 'Not Heard'

        # 2. Denied
        elif 'denied' in s:
            return 'Denied'

        # 3. Granted
        elif ('granted' in s):
            return 'Granted'

        # 4. Heard
        elif ('heard' in s):
            return 'Heard'

        # 5. Anything Else
        else:
            return 'Other'
    else:
      return None
  #except Exception as e: print(s, e)

# Map statuses to predefined categories
events['result'] = events['result'].apply(map_motion_results)

In [13]:
events.head(50)

,case_id,location,date,type,result,official
0,24CR34186,Department 3,NaT,Arraignment: Complaint,Other,PAK
2,24CR34185,Department 3,NaT,Arraignment: Complaint,Other,PAK
4,24CR34184,Department 1,NaT,Arraignment: Complaint,Other,WHITE
6,24CR34183,Department 3,NaT,Arraignment: Complaint,Other,PAK
8,24CR34182,Department 3,NaT,Arraignment: Citation,Other,PAK
10,24CR34181,Department 3,NaT,Arraignment: Complaint,Other,PAK
12,24CR34180,Department 3,NaT,Arraignment: Citation,Other,PAK
14,24CR34179,Department 3,NaT,Arraignment: Complaint,Other,PAK
16,24CR34178,Department 3,NaT,Arraignment: Complaint,Other,PAK
18,24CR34177,Department 3,NaT,Further Arraignment: Complaint,Other,PAK


## Charges

In [14]:
# Handle NA Columns
charges = drop_empty(charges)
charges.rename(columns={'case_name': 'charge'}, inplace=True)

# Remove empty duplicate rows
charges = charges[~(charges['charge'].isna() & charges['disposition'].isna())]

# Remove names from Charge column
charges = charges[~charges['charge'].str.match(r'^[A-Z\s,]+(?:Jr)?$')]

charges['disposition'] = charges['disposition'].str.split('; ', n=1).str[-1]

# Extract the disposition text and a trailing date if present
# The pattern matches any text followed by a date in MM/DD/YYYY format at the end
extracted = charges['disposition'].str.extract(r'^(.*?)(\d{2}/\d{2}/\d{4})$')

# Assign the extracted date to a new column
charges['disp_date'] = pd.to_datetime(extracted[1], errors='coerce')

# If no date was found, fallback to original disposition; otherwise, use the extracted text
charges['disposition'] = extracted[0].fillna(charges['disposition']).str.strip()

C:\Users\rubyc\AppData\Local\Temp\ipykernel_2448\1030528905.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace(to_replace={"": np.nan}, inplace=True)


## Documents

In [15]:
# Handle NA Columns
documents = drop_empty(documents)

# Normalize Columns
documents['filed_by'] = documents['filed_by'].str.split(' -', n=1).str[0]

# Normalize Dates
documents['filed_date'] = documents['filed_date'].str.extract(r'^(\d{2}/\d{2}/\d{4})')
documents['filed_date'] = pd.to_datetime(documents['filed_date'], errors='coerce', format='%m/%d/%Y')

documents['tracking_date'] = documents['tracking_date'].str.extract(r'(\d{2}/\d{2}/\d{4})')
documents['tracking_date'] = pd.to_datetime(documents['tracking_date'], errors='coerce', format='%m/%d/%Y')

# Retain rows with information
documents = documents[~(documents['filed_date'].isna() & documents['filed_by'].isna() & documents['description'].isna() & documents['tracking_date'].isna())]

# Indigent Defense Indicator
oapd_mask = documents['description'].str.contains('OAPD', case=False, na=False)
case_ids_with_oapd = documents.loc[oapd_mask, 'case_id'].unique()
documents['indigent_defense'] = documents['case_id'].isin(case_ids_with_oapd)

C:\Users\rubyc\AppData\Local\Temp\ipykernel_2448\1030528905.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace(to_replace={"": np.nan}, inplace=True)


In [16]:
documents.head()

,case_id,filed_date,description,filed_by,tracking_date,indigent_defense
0,24CR34186,NaT,Notice of Hearing,Clerk,2024-09-18,False
1,24CR34186,2024-09-18,Peremptory Challenge,District Attorney,NaT,False
2,24CR34186,2024-09-18,Complaint,District Attorney,2024-09-18,False
4,24CR34185,NaT,Document: Other TBD,Clerk,NaT,False
5,24CR34185,2024-09-18,Complaint,District Attorney,2024-09-18,False


In [17]:
documents.indigent_defense.value_counts()

indigent_defense
True     315787
False    262081
Name: count, dtype: int64

## Bail

In [18]:
# Handle NA Columns
bail = drop_empty(bail)

# Rename bail_date to set_date
bail.rename(columns={'bail_date': 'set_date'}, inplace=True)

# Extract the dollar amount and clean it up
bail['amount'] = bail['bail_type_status'].str.extract(r'\$(\d[\d,]*)\.')
bail['amount'] = bail['amount'].str.replace(',', '', regex=False)
bail['amount'] = pd.to_numeric(bail['amount'], errors='coerce')
# Extract posted_date
bail['posted_date'] = bail['bail_type_status'].str.extract(r'Posted:\s(\d{2}/\d{2}/\d{4})')[0]

# posted is True if "posted" is found and "not posted" is not found, case-insensitive
bail['posted'] = (bail['bail_type_status'].str.contains('posted', case=False, na=False) &
                  ~bail['bail_type_status'].str.contains('not posted', case=False, na=False))

# Convert dates to datetime
bail['set_date'] = pd.to_datetime(bail['set_date'], errors='coerce', format='%m/%d/%Y')
bail['posted_date'] = pd.to_datetime(bail['posted_date'], errors='coerce', format='%m/%d/%Y')

# Drop bail_type_status
bail.drop(columns=['bail_type_status', 'bail_posted_by'], inplace=True)

# Consolidate rows for same case_id
bail = bail.groupby('case_id').agg({
    'set_date': 'first',
    'posted_date': 'first',
    'amount': 'max',
    'posted': 'max'
}).reset_index()

C:\Users\rubyc\AppData\Local\Temp\ipykernel_2448\1030528905.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace(to_replace={"": np.nan}, inplace=True)


## Probation

In [19]:
# Handle NA Columns
probation = drop_empty(probation)
probation_data = probation.copy()

# Assuming the format: "Probation: (Formal|Summary) X (Months|Years) Granted on MM/DD/YYYY Imposed as of MM/DD/YYYY"
pattern = r'Probation:\s+(?:Formal|Summary)\s+(\d+)\s+(Months|Years)\s+Granted\s+on\s+(\d{2}/\d{2}/\d{4})\s+Imposed\s+as\s+of\s+(\d{2}/\d{2}/\d{4})'

extracted = probation_data['probation_conditions'].str.extract(pattern)

# Convert duration to numeric, handle years by multiplying by 12
extracted[0] = extracted[0].astype(float)
extracted['probation_length'] = extracted.apply(lambda x: x[0]*12 if x[1] == 'Years' else x[0], axis=1)

# Assign new columns from extracted data
probation_data['probation_length'] = extracted['probation_length']
probation_data['probation_granted_date'] = pd.to_datetime(extracted[2], errors='coerce', format='%m/%d/%Y')
probation_data['probation_imposed_date'] = pd.to_datetime(extracted[3], errors='coerce', format='%m/%d/%Y')

# Propagate these values to all rows of the same case_id
probation_data[['probation_length', 'probation_granted_date', 'probation_imposed_date']] = (
    probation_data.groupby('case_id')[['probation_length', 'probation_granted_date', 'probation_imposed_date']]
    .transform(lambda grp: grp.ffill().bfill())
)

# Drop the original probation_conditions column now that we have new columns
probation_data.drop(columns='probation_conditions', inplace=True)

# Drop rows where probation_type starts with "Defendant"
probation_data = probation_data[~probation_data['probation_type'].str.startswith('Defendant', na=False)]

# Rename Columns
probation_data.rename(columns={'probation_type': 'probation_conditions', 'probation_length': 'length_months', 'probation_granted_date': 'date_granted', 'probation_imposed_date': 'date_imposed'}, inplace=True)

# Retain non-NA relevant rows for probation_condition where there are other rows for that same case id for with not na probation_conditions
probation = probation_data[
    ~(
        probation_data['probation_conditions'].isna() &
        probation_data.groupby('case_id')['probation_conditions'].transform('any').notna()
    )
]

C:\Users\rubyc\AppData\Local\Temp\ipykernel_2448\1030528905.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace(to_replace={"": np.nan}, inplace=True)


## Consolidating, Ordering, and Finalizing

In [20]:
# Swapping indigent defense indicator from documents to cases
indigent_defense_map = documents.groupby('case_id')['indigent_defense'].any()
cases['indigent_defense'] = cases['case_id'].map(indigent_defense_map).fillna(False).astype(bool)
documents.drop(columns=['indigent_defense'], inplace=True)

# Adding represented_by to cases merged on case_id from parties where type == 'Defendant'
cases = pd.merge(cases, parties[parties['type'] == 'Defendant'][['case_id', 'represented_by']], on='case_id', how='left')
parties.drop(columns='represented_by', inplace=True)

# Reset indices of all data frames
cases.reset_index(drop=True, inplace=True)
parties.reset_index(drop=True, inplace=True)
events.reset_index(drop=True, inplace=True)
charges.reset_index(drop=True, inplace=True)
documents.reset_index(drop=True, inplace=True)
bail.reset_index(drop=True, inplace=True)
probation.reset_index(drop=True, inplace=True)

C:\Users\rubyc\AppData\Local\Temp\ipykernel_2448\341180305.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  cases['indigent_defense'] = cases['case_id'].map(indigent_defense_map).fillna(False).astype(bool)


In [21]:
# Standardizing Representation

# Standardizing Representation
representation_mapping = {
    # Deduplication
    "ASBELL, APRIL ASBELL, APRIL": "ASBELL, APRIL",
    "HOLBUS, JOHN W HOLBUS, JOHN W": "HOLBUS, JOHN W",
    "SEATON, JEFFREY SEATON, JEFFREY": "SEATON, JEFFREY",
    "CRAMER, DAVID JEFFREY CRAMER, DAVID JEFFREY": "CRAMER, DAVID JEFFREY",

    # Multi-Name Consolidation
    "ASBELL, APRIL FLEMING, LEIGH": "ASBELL, APRIL; FLEMING, LEIGH",
    "FLEMING, LEIGH ASBELL, APRIL": "ASBELL, APRIL; FLEMING, LEIGH",
    "SCOTT, APRIL FLEMING, LEIGH": "SCOTT, APRIL; FLEMING, LEIGH",
    "SCOTT, APRIL ADAMSON, KEVIN FLEMING, LEIGH ADAMSON, KEVIN": "SCOTT, APRIL; FLEMING, LEIGH; ADAMSON, KEVIN",
    "FLEMING, LEIGH ASBELL, APRIL DAWN": "ASBELL, APRIL; FLEMING, LEIGH",
    "ASBELL, APRIL DAWN FLEMING, LEIGH": "ASBELL, APRIL; FLEMING, LEIGH",

    # Case Sensitivity
    "Lindemann, Callie": "LINDEMANN, CALLIE",

    # Firm Overlaps
    "CIUMMO, RICHARD A FITZGERALD ALVAREZ & CIUMMO": "FITZGERALD ALVAREZ & CIUMMO",
    "SCHELL, ROBERT CHARLES FITZGERALD ALVAREZ & CIUMMO": "FITZGERALD ALVAREZ & CIUMMO",

    # Firm Variations
    "FITZGERALD ALVAREZ & CIUMMO FITZGERALD ALVAREZ & CIUMMO": "FITZGERALD ALVAREZ & CIUMMO",
    "FITZGERALD, ALVAREZ AND CIUMMO": "FITZGERALD ALVAREZ & CIUMMO",
    "FITZGERALD, ALVAREZ AND CUIMMO": "FITZGERALD ALVAREZ & CIUMMO",
    "FITZGERALD ALVAREZ & CUIMMO": "FITZGERALD ALVAREZ & CIUMMO",

    # Misspellings
    "WEINER, DAIN WEINER, DAVIA": "WEINER, DAIN",
    "TIEMANN, ROLAND X BEREZIN, MICHAEL": "TIEMANN, ROLAND X; BEREZIN, MICHAEL",
    "TIEMANN, ROLAND X CIUMMO, RICHARD A": "TIEMANN, ROLAND X; CIUMMO, RICHARD A",
    "BARKER, JOHN A CIUMMO, RICHARD A": "CIUMMO, RICHARD A; BARKER, JOHN A",
    "CIUMMO, RICHARD A BARKER, JOHN A": "CIUMMO, RICHARD A; BARKER, JOHN A",
    "CRAMER, DAVID JEFFREY FITZGERALD ALVAREZ & CIUMMO": "FITZGERALD ALVAREZ & CIUMMO; CRAMER, DAVID JEFFREY",
    "FLEMING, LEIGH FITZGERALD ALVAREZ & CIUMMO": "FITZGERALD ALVAREZ & CIUMMO; FLEMING, LEIGH",
    "WEINER, DAIN WEINER, DAVID": "WEINER, DAIN; WEINER, DAVID",
    "SCOTT, APRIL; FLEMING, LEIGH": "SCOTT, APRIL; FLEMING, LEIGH",
    "SCOTT, APRIL; FLEMING, LEIGH; ADAMSON, KEVIN": "SCOTT, APRIL; FLEMING, LEIGH; ADAMSON, KEVIN",
}

# Apply the mapping to the 'represented_by' column
cases['represented_by'] = cases['represented_by'].replace(representation_mapping)

In [ ]:
keyword_dict = {
    "PD": ['FITZGERALD ALVAREZ & CIUMMO', 'PUBLIC DEFENDER', "CIUMMO, RICHARD A"],
    "Wheel": ["ASBELL, APRIL", "FLEMING, LEIGH", "LINDEMANN, CALLIE", "CRAMER, DAVID JEFFREY"]
}

def pd_sort(df): 

    # Helper function for keyword matching
    def match_keywords(text, keyword_dict):
        for category, keywords in keyword_dict.items():
            if pd.notnull(text) and any(keyword in text for keyword in keywords):
                return category
        return None

    # Process each row
    def process_row(row):
        pd_status = "Unknown"
        
        if pd.notnull(row['represented_by']):
            pd_status = match_keywords(row['represented_by'], keyword_dict) or None
        else:
            pd_status = None

        return pd_status

    # Apply the processing function
    df[['pd_status']] = df.apply(lambda row: pd.Series(process_row(row)), axis=1)

    return df


In [47]:
pd_sort(cases)

,case_id,case_name,filing_date,location,case_type,category,status,update_date,indigent_defense,represented_by,pd_status
0,24CR34186,People vs. YESSENIA ESCOBAR ROCHA,2024-09-18,Criminal/Traffic,Misdemeanor,Misdemeanor: Drug Offense,Open,NaT,False,NaN,NaN
1,24CR34185,"People vs. CESAR JUNIOR MORENO, Jr",2024-09-18,Criminal/Traffic,Misdemeanor,Misdemeanor: DUI,Open,NaT,False,NaN,NaN
2,24CR34184,People vs. COURTNEY JONATHAN GARLAND,2024-09-18,Criminal/Traffic,Misdemeanor,Misdemeanor: Non Traffic/Ordinance,Open,NaT,False,NaN,NaN
3,24CR34183,People vs. BENJAMIN WILSON HOLCOMB,2024-09-17,Criminal/Traffic,Misdemeanor,Misdemeanor: Traffic Other,Open,NaT,False,NaN,NaN
4,24CR34182,People vs. JOSHUA JAMES BANNON,2024-09-17,Criminal/Traffic,Misdemeanor,Misdemeanor: 14601VC,Open,NaT,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
33547,02-CR-01806,People vs. LOLA BELL TURLEY,2002-05-29,Criminal/Traffic,Misdemeanor,Misdemeanor: 14601VC,Conviction — Nolo Plea,2005-03-09,False,NaN,NaN
33548,02-CR-01805,People vs. THOMAS OTTO MOSELEY,2002-05-29,Criminal/Traffic,Misdemeanor,Misdemeanor: 14601VC,Dismissed,2002-08-28,False,NaN,NaN
33549,06-CR-11297,People vs. TROY TOWNER,2006-09-25,Criminal/Traffic,Misdemeanor,Misdemeanor: DUI,Dismissed,2006-10-16,False,NaN,NaN
33550,06-CR-11342,People vs. RICH JAMES TELLER,2006-10-05,Criminal/Traffic,Misdemeanor,Misdemeanor: Assault and Battery,Conviction — Nolo Plea,2006-11-02,True,"CIUMMO, RICHARD A",PD


In [22]:
pd.DataFrame(cases['represented_by'].value_counts())

,count
represented_by,
FITZGERALD ALVAREZ & CIUMMO,2017
IN PRO PER,1412
"CIUMMO, RICHARD A",251
"ASBELL, APRIL; FLEMING, LEIGH",227
"SINGER, DAVID K",126
...,...
"CIOCCA, PATRICK MICHAEL",1
"BENAVIDEZ, ARTURO",1
PORTANOVA,1


# Codebook

## Cases Table
| Column Name          | Description                                               |
|----------------------|-----------------------------------------------------------|
| case_id              | Unique identifier for the court case.                     |
| case_name            | Title of the case, including parties involved.            |
| filing_date          | Date the case was filed.                                  |
| location             | Court location or division handling the case.             |
| case_type            | Broad classification of the case (e.g., criminal, traffic). |
| category             | Specific category or description of the case type.        |
| status               | Current status of the case (e.g., Open, Closed).          |
| update_date          | Date of the last update to the case record.               |
| indigent_defense     | Indicates if a public defender has been assigned (True/False). |
| represented_by       | Name or entity representing the defendant.                |

## Parties Table
| Column Name          | Description                                               |
|----------------------|-----------------------------------------------------------|
| case_id              | Unique identifier for the court case.                     |
| type                 | Role of the individual in the case (e.g., Defendant).     |
| name                 | Name of the individual involved in the case.              |

## Events Table
| Column Name          | Description                                               |
|----------------------|-----------------------------------------------------------|
| case_id              | Unique identifier for the court case.                     |
| location             | Department or courtroom where the event occurred.         |
| date                 | Date of the event.                                        |
| type                 | Description of the event (e.g., Arraignment, Hearing).    |
| official             | Last name of the judge presiding over the event.          |

## Charges Table
| Column Name          | Description                                               |
|----------------------|-----------------------------------------------------------|
| case_id              | Unique identifier for the court case.                     |
| charge               | Description of the charge filed in the case.              |
| disposition          | Final disposition or outcome of the charge.               |
| disp_date            | Date the disposition was issued.                          |

## Documents Table
| Column Name          | Description                                               |
|----------------------|-----------------------------------------------------------|
| case_id              | Unique identifier for the court case.                     |
| filed_date           | Date the document was filed in the case.                  |
| description          | Description of the document filed.                        |
| filed_by             | Party responsible for filing the document (e.g., Clerk, DA). |
| tracking_date        | Date for tracking the document's status.                  |

## Probation Table
| Column Name          | Description                                               |
|----------------------|-----------------------------------------------------------|
| case_id              | Unique identifier for the court case.                     |
| probation_conditions | Conditions assigned for the probation period.             |
| length_months        | Length of the probation period in months.                 |
| date_granted         | Date the probation was granted.                           |
| date_imposed         | Date the probation was imposed.                           |

## Bail Table
| Column Name          | Description                                               |
|----------------------|-----------------------------------------------------------|
| case_id              | Unique identifier for the court case.                     |
| set_date             | Date bail was set.                                        |
| posted_date          | Date bail was posted, if applicable.                      |
| amount               | Amount of bail set for the case.                          |
| posted               | Indicates if the bail was posted (True/False).            |

# Analysis

## Completeness

In [23]:
# Checking uniqueness of case_id in each table
tables = {'cases': cases, 'parties': parties, 'events': events,
          'charges': charges, 'documents': documents,
          'probation': probation, 'bail': bail}

for table_name, df in tables.items():
    is_unique = df['case_id'].is_unique
    print(f"Is 'case_id' unique in {table_name}? {is_unique}")

Is 'case_id' unique in cases? True
Is 'case_id' unique in parties? False
Is 'case_id' unique in events? False
Is 'case_id' unique in charges? False
Is 'case_id' unique in documents? False
Is 'case_id' unique in probation? False
Is 'case_id' unique in bail? True


In [24]:
# Check case_id consistency across tables
main_cases = set(cases['case_id'])
for table_name, df in tables.items():
    if table_name != 'cases':
        other_cases = set(df['case_id'])
        missing = other_cases - main_cases
        print(f"Missing case_ids in {table_name} not found in cases table: {missing}")

Missing case_ids in parties not found in cases table: set()
Missing case_ids in events not found in cases table: set()
Missing case_ids in charges not found in cases table: set()
Missing case_ids in documents not found in cases table: set()
Missing case_ids in probation not found in cases table: set()
Missing case_ids in bail not found in cases table: set()


In [25]:
# Check for missing values in each table
for table_name, df in tables.items():
    print(f"\nMissing values in {table_name} table:")
    print(df.isnull().sum())


Missing values in cases table:
case_id                 0
case_name               0
filing_date            17
location                0
case_type               0
category                0
status                 59
update_date          3845
indigent_defense        0
represented_by      28412
dtype: int64

Missing values in parties table:
case_id    0
type       0
name       0
dtype: int64

Missing values in events table:
case_id         0
location     5501
date        32563
type            2
result          0
official     1268
dtype: int64

Missing values in charges table:
case_id            0
charge             0
disposition    59849
disp_date      61501
dtype: int64

Missing values in documents table:
case_id               0
filed_date        33373
description           0
filed_by         530074
tracking_date    551667
dtype: int64

Missing values in probation table:
case_id                    0
probation_conditions       0
length_months           9356
date_granted            9356
dat